In [34]:
# pip install --upgrade --user ortools

In [35]:
from PointerNet import PtrNet, Critic
from tsp_data import Tsp
import torch

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

import time
from tqdm import tqdm
import matplotlib.pyplot as plt

In [36]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

cuda


In [37]:
# Load (Generate) dataset

# tsp = Tsp(N=50)
tsp = Tsp(N=136)
# X, _ = tsp.next_batch(1280000)
X = torch.load("tsp100_instances.pt").to(device)

X

tensor([[[0.9113, 5.5867],
         [5.6572, 5.8759],
         [7.8759, 8.3096],
         ...,
         [6.3746, 0.0212],
         [2.0425, 2.4690],
         [7.6219, 2.2999]],

        [[7.8990, 3.8796],
         [3.3220, 8.3968],
         [7.0573, 2.2850],
         ...,
         [1.3557, 9.7055],
         [1.6539, 0.9246],
         [6.4377, 9.3052]],

        [[5.3749, 5.5163],
         [1.5314, 9.3630],
         [1.5162, 7.9102],
         ...,
         [9.7241, 5.7229],
         [4.7208, 4.2310],
         [6.9921, 6.1235]],

        ...,

        [[6.2131, 1.1924],
         [6.4167, 8.3115],
         [1.6980, 6.3862],
         ...,
         [5.6985, 3.5763],
         [9.9159, 7.7913],
         [8.5293, 9.5337]],

        [[9.6788, 2.5420],
         [4.9812, 9.2833],
         [5.5906, 4.7701],
         ...,
         [7.6115, 6.7717],
         [7.7042, 9.5903],
         [7.8305, 4.9100]],

        [[0.2710, 4.9755],
         [3.4787, 1.2458],
         [0.1557, 9.5829],
         ...,
 

In [38]:
X = torch.LongTensor
X_val,_ = tsp.next_batch(1000)
X_val = X_val.to(device)

In [40]:
print(type(X))

<class 'torch.tensortype'>


In [39]:
# Make dataloader

dataset = TensorDataset(X)
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

TypeError: descriptor 'size' for 'torch._C._TensorBase' objects doesn't apply to a 'int' object

In [ ]:
# Initialize model
hidden_dim = 256
model = PtrNet(hidden_dim=hidden_dim).to(device)
critic = Critic(hidden_dim=hidden_dim).to(device)


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 530,688 trainable parameters


In [ ]:
Learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=Learning_rate)
optimizer_ = optim.Adam(critic.parameters(), lr=Learning_rate)

In [ ]:
Validation = []

print(f"...Start training on {device}...")
for epoch in range(10):
    for i, s_i in tqdm(enumerate(dataloader)):
        s_i = s_i[0]
        p, pi = model(s_i)
        L = model.get_length(s_i,pi)
        b = critic(s_i)
        
        with torch.no_grad():
            advantage = L - b
        
        loss = torch.mean(advantage * p)
        loss_ = torch.mean((L - b)**2)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        optimizer_.zero_grad()
        loss_.backward()
        optimizer_.step()

        if (i + 1)%1000 == 0:
            _, pi_val = model(X_val)
            L_val = model.get_length(X_val,pi_val)
            Validation.append(torch.mean(L_val).item())
            print(f"Mean tour length for validation set at {(i+1)*(epoch+1)}th step: {torch.mean(L_val):.2f}")
    

...Start training on cuda...


NameError: name 'dataloader' is not defined

In [ ]:
plt.plot(Validation)

In [ ]:
# torch.save(model.state_dict(), 'tsp_100_model_weigths.pth')

In [ ]:
tsp = Tsp(N=45)
x,_ = tsp.next_batch(1)

In [ ]:
_,sol = model(x.to(device))

In [ ]:
def plot_tsp_solution(coords, tour):
    """
    coords: (num_cities, 2) - 각 도시의 좌표
    tour: (num_cities,) - 선택된 도시의 인덱스로 구성된 투어
    """
    num_cities = len(coords)
    
    # 경로 좌표 추출
    path_coords = [coords[i] for i in tour]
    
    # 경로를 닫힌 형태로 만들기 위해 시작 도시를 경로에 추가
    path_coords.append(coords[tour[0].item()])
    
    # 좌표 추출
    path_x = [coord[0].item() for coord in path_coords]
    path_y = [coord[1].item() for coord in path_coords]
    
    # 그래프 그리기
    plt.figure(figsize=(6, 6))
    plt.plot(path_x, path_y, 'o-')
    plt.scatter(path_x[0], path_y[0], marker='s', color='red', label='Start')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('TSP Solution')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_tsp_solution(x[0],sol[0])

In [ ]:
sol[0][0].item()